In [1]:
import tweepy #https://github.com/tweepy/tweepy
import csv

DEBUG = False

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    user=api.get_user(screen_name=screen_name)
    
    #retweets=([t.user.screen_name for t in new_tweets[0].retweets()])
    
    #return
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        if DEBUG: print "getting tweets before %s" % (oldest)
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,
                                       count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        if DEBUG: print "...%s tweets downloaded so far" % (len(alltweets))
    
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [{'id_str' : tweet.id_str, 
                  'created_at' : tweet.created_at, 
                  'text' : tweet.text.encode("utf-8"),
                  'retweet_count' : tweet.retweet_count,
                  'screen_name' : screen_name}
                 
                 for tweet in alltweets]
    
    
    return outtweets
    
    #write the csv    
#     with open('%s_tweets.csv' % screen_name, 'wb') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text", "retweet_count", "retweet_user_names"])
#         writer.writerows(outtweets)
    
#     pass

tweets = get_all_tweets("kaskade")

In [2]:
len(tweets)

3190

In [3]:
original_tweets = [t for t in tweets if t['text'][:2] != 'RT']

In [4]:
len(original_tweets)

2160

In [5]:
ids = [tw["id_str"] for tw in original_tweets]

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
new_ids=[]
for i in ids:
    new_ids.append(i.encode('utf-8'))
new_ids


['836984179338272768',
 '836983727519633408',
 '836983470299758592',
 '836968520772636674',
 '836730654394040321',
 '836652187061211137',
 '836651216713187328',
 '836637277946269696',
 '836635582218514432',
 '836633485024571394',
 '836625647120318464',
 '836618580703735808',
 '836362361875415042',
 '836361300808773633',
 '836291606013706241',
 '836288268702482433',
 '836274216445976577',
 '836270254724681729',
 '836260702088093698',
 '836258915620958208',
 '836241672023461888',
 '835976061074702336',
 '835973532345634817',
 '835628541366632448',
 '835621632341323776',
 '835326494415159296',
 '835249699359756288',
 '834861561642614788',
 '834846536827805697',
 '834816539048046593',
 '834561519069384704',
 '834560919669710848',
 '834547193373618176',
 '834544030927065088',
 '834503493130059777',
 '834493085581090817',
 '834461216181538817',
 '834460588512251905',
 '834460046641811463',
 '834306391900712960',
 '834305268670296064',
 '834303865746907136',
 '834303103872569345',
 '834302487

In [8]:
import time

In [9]:
len(new_ids)

2160

In [26]:
def id_sn_location(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
            
        with open('kaskade300.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [27]:
#id_sn_location(new_ids[0:300])

In [28]:
def id_sn_location1000(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
            
        with open('kaskade1000.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [29]:
#id_sn_location1000(new_ids[1001:])

In [30]:
import pandas as pd

In [31]:
kaskade=pd.read_csv('kaskade300.csv')
kaskade301=pd.read_csv('kaskade2.csv')
kaskade1000=pd.read_csv('kaskade1000.csv')

In [32]:
kaskade.head()

id     screen_name           location
0  836984179338272768     iamclaudia_                NaN
1  836984179338272768        emilymxj        Chicago, IL
2  836984179338272768  LilEmpireMusic                 LA
3  836984179338272768     adasoftwaer  Frankfurt am Main
4  836984179338272768   TheKingDustin         Skokie, IL

In [33]:
kaskade301.head()

id     screen_name                  location
0  810692530555723776  2Pac_CPoSaDa12  United States of America
1  810692530555723776     _IAmKristen                       NaN
2  810692530555723776        taterrzz                        TX
3  810692530555723776       genzepeda               Houston, TX
4  810692530555723776       A_Elsaadi                       HTX

In [34]:
kaskade1000.head()

id    screen_name          location
0  755486818410397696   SaurabhSwift  Madhepura, India
1  755486818410397696  hollymostdope      Sorrento, FL
2  755486818410397696   bythethroaat        South Bay 
3  755486818410397696  Lovely_Lex101               NaN
4  755486818410397696     Dwayne8id3       Vernal UTAH

In [93]:
kaskade2=[kaskade, kaskade301, kaskade1000]

In [94]:
thekaskade=pd.concat(kaskade2)
thekaskade

id      screen_name                    location
0      836984179338272768      iamclaudia_                         NaN
1      836984179338272768         emilymxj                 Chicago, IL
2      836984179338272768   LilEmpireMusic                          LA
3      836984179338272768      adasoftwaer           Frankfurt am Main
4      836984179338272768    TheKingDustin                  Skokie, IL
5      836984179338272768    LiveNationPNW                 Seattle, WA
6      836984179338272768        jennyn___                         NaN
7      836984179338272768         p_man_74                         NaN
8      836984179338272768  MirandaNico6le3                    New York
9      836984179338272768   LadyLindsayAnn        Miami Beach, Florida
10     836984179338272768  DuckwjrthInsreu                    New York
11     836984179338272768        Ramos__98                Kaskade Land
12     836984179338272768       KARLECAKES                Honolulu, HI
13     836984179338272768    frankieshakes             Toronto, Canada
14     836984179338272768  jessicadiane198        Great state of Texas
15     836984179338272768        danman022                     Chicago
16     836984179338272768       ryanimal34                     Chicago
17     836984179338272768     therealjnepo        Let the music speak.
18     836984179338272768        kashkad13                 Chicago, IL
19     836983727519633408  MirandaNico6le3                    New York
20     836983727519633408  DuckwjrthInsreu                    New York
21     836983727519633408    oceanicglider               Las Vegas, NV
22     836983727519633408        JeremyM09           still at work, tx
23     836983470299758592      fersolis011                         NaN
24     836983470299758592  MirandaNico6le3                    New York
25     836983470299758592  DuckwjrthInsreu                    New York
26     836983470299758592       isabell1_d                      Brasil
27     836983470299758592       Daves2Dope                         NaN
28     836983470299758592        JeremyM09           still at work, tx
29     836983470299758592        MSgtPanda                  Senoia, GA
...                   ...              ...                         ...
17505  686982572308578304     itsgraaaciie                          TX
17506  686982572308578304        geoxperez           Riverside/East LA
17507  686982572308578304     lesleysmalls                         NaN
17508  686982572308578304         leflower             California, USA
17509  686982572308578304          ReyJayy                     phx, az
17510  686982572308578304           m_bnzz             Los Angeles, CA
17511  686982572308578304       LaEdcRaver                         NaN
17512  686982572308578304        _Charlie5       South Los Angeles, CA
17513  686982572308578304      jennniferk_                          LA
17514  686982572308578304   ricardo_avitia                         NaN
17515  686982572308578304        cassy_htx                 Houston, TX
17516  686982572308578304       kaypeelove                  Walnut, CA
17517  686982572308578304          sgtsltr                     Basspod
17518  686982572308578304       Chikita323              Los Angeles CA
17519  686981454748893184        Oskeylp22                       (213)
17520  686981454748893184      charleysuxx             California, USA
17521  686981454748893184      _floresbeto                         NaN
17522  686981454748893184         leflower             California, USA
17523  686981454748893184        geoxperez           Riverside/East LA
17524  686981454748893184     lesleysmalls                         NaN
17525  686981454748893184          ReyJayy                     phx, az
17526  686981454748893184     thatsmejesus  Glassell Park, Los Angeles
17527  686981454748893184  shackcanterbury            Canterbury, Kent
17528  686981454748893184      airandstyle                         NaN
17529  686981454748893184       LaEdcRaver     

In [96]:
thekaskade.to_pickle('kaskade.pkl')

In [95]:
len(thekaskade['id'].unique())

2158

In [38]:
test=thekaskade.groupby(['id'], as_index=False).agg(lambda x: set(x))

In [39]:
test2=test.drop(['location'], axis=1)

In [40]:
test3=test2['screen_name'].str.join(sep='*').str.get_dummies(sep='*')

In [41]:
test4=pd.concat([test2, test3], axis=1)

In [42]:
test4.head()

id                                        screen_name  \
0  686981454748893184  {lesleysmalls, leflower, aYoBrandon, bryanaeri...   
1  686982572308578304  {ricardo_avitia, lesleysmalls, xxxdanielcd, le...   
2  686995727579496448  {CaaatchMyDrift, MrPartyyyHardy, DECO2akaSEA, ...   
3  687073144948011008  {plsdre, misakimemii, fluffingkendall, samfeld...   
4  687306427782803456  {funxjjh, DJVinnyVice, ajk_jp, samfeldt_, waiw...   

   005nightrdire  04N1374  05rd17o  0718Gomez  07Adeakm  09210info  09moCza  \
0              0        0        0          0         0          0        0   
1              0        0        0          0         0          0        0   
2              0        0        0          0         0          0        0   
3              0        0        0          0         0          0        0   
4              0        0        0          0         0          0        0   

   0Alber0       ...         zp_pidoxpr  zshooter  zufars  zulixo  \
0        0       ...                  0         0       0       0   
1        0       ...                  0         0       0       0   
2        0       ...                  0         0       0       0   
3        0       ...                  0         0       0       0   
4        0       ...                  0         0       0       0   

   zulmastillwlhel  zurizadays  zyiarreee  zymappso  zyu_chan3020  \
0                0           0          0         0             0   
1                0           0          0         0             0   
2                0           0          0         0             0   
3                0           0          0         0             0   
4                0           0          0         0             0   

   zz6AcwyYmGT35wC  
0                0  
1                0  
2                0  
3                0  
4                0  

[5 rows x 18698 columns]

In [43]:
test4.sum(axis=0)

id                 5533345123103970518
005nightrdire                        1
04N1374                              1
05rd17o                              1
0718Gomez                            1
07Adeakm                             2
09210info                            1
09moCza                              1
0Alber0                              1
0ImJP0                               1
0ceanicskies                         1
0p_ish                               7
0rb0tn1nja                           1
0rthoaead                            1
0ver_and0ut                          1
100AMPrtcjeo                         2
102375Brandon                        2
1035NPSE                             2
103Visuass                           1
1099rock                             1
10Hastin6sg6                         2
10Pips_Anyimet                       1
10by10Evnset                         2
11_armijo                            1
11alexbsnru                          1
12oiner                  

In [44]:
test4.sum(axis=0).sort_values(ascending=False)

id                 5533345123103970518
Kaskade_MX                         299
FedeeeeOK                          236
BhuvaneshBp                        195
MomentOfChaos                      167
oceanicglider                      128
WhenInRome_18                       98
savageslitz                         83
ZhaoChuen                           75
misakimemii                         74
edmchic_nadz                        72
NoreenTranceCat                     65
SaurabhSwift                        55
funnybunny640                       51
pinguina_23                         49
hollymostdope                       46
nervo_colombia                      45
89623699167                         44
bigbootydoobie                      40
paulosj16                           39
_huertta                            37
Minichyna01                         36
mikey929198555                      35
justgema                            35
dabxrave                            34
SHSoni_v1                

In [45]:
tweets2 = get_all_tweets("Kaskade_MX")
tweets3 = get_all_tweets("FedeeeeOK")
tweets4 = get_all_tweets("BhuvaneshBp")

In [46]:
tweets5 = get_all_tweets("MomentOfChaos")
tweets6 = get_all_tweets("oceanicglider")
tweets7 = get_all_tweets("WhenInRome_18")

In [47]:
tweets8 = get_all_tweets("savageslitz")
tweets9 = get_all_tweets("ZhaoChuen")
tweets10 = get_all_tweets("misakimemii")
tweets11 = get_all_tweets("edmchic_nadz")

In [48]:
print len(tweets2)
print len(tweets3)
print len(tweets4)
print len(tweets5)
print len(tweets6)
print len(tweets7)
print len(tweets8) 
print len(tweets9) 
print len(tweets10)
print len(tweets11)

3200
3205
3226
3245
3199
3188
3229
3203
3246
3226


In [49]:
ogt1 = [t for t in tweets2 if t['text'][:2] != 'RT'] 
ogt2 = [t for t in tweets3 if t['text'][:2] != 'RT']
ogt3 = [t for t in tweets4 if t['text'][:2] != 'RT']
ogt4 = [t for t in tweets5 if t['text'][:2] != 'RT']
ogt5 = [t for t in tweets6 if t['text'][:2] != 'RT']
ogt6 = [t for t in tweets7 if t['text'][:2] != 'RT']
ogt7 = [t for t in tweets8 if t['text'][:2] != 'RT']
ogt8 = [t for t in tweets9 if t['text'][:2] != 'RT']
ogt9 = [t for t in tweets10 if t['text'][:2] != 'RT']
ogt10 = [t for t in tweets11 if t['text'][:2] != 'RT']

In [51]:
print len(tweets2), len(ogt1) 
print len(tweets3), len(ogt2) 
print len(tweets4), len(ogt3) 
print len(tweets5), len(ogt4) 
print len(tweets6), len(ogt5) 
print len(tweets7), len(ogt6) 
print len(tweets8) , len(ogt7) 
print len(tweets9) , len(ogt8) 
print len(tweets10), len(ogt9) 
print len(tweets11), len(ogt10)

3200 739
3205 850
3226 4
3245 976
3199 1406
3188 593
3229 363
3203 3
3246 113
3226 264


In [52]:
kaskade_tweets=pd.DataFrame(original_tweets)
kaskade_tweets.head()

created_at              id_str  retweet_count screen_name  \
0 2017-03-01 16:59:13  836984179338272768              3     kaskade   
1 2017-03-01 16:57:25  836983727519633408              2     kaskade   
2 2017-03-01 16:56:24  836983470299758592              3     kaskade   
3 2017-03-01 15:57:00  836968520772636674             11     kaskade   
4 2017-03-01 00:11:48  836730654394040321             11     kaskade   

                                                text  
0  You + me + a #SpringFling! PRESALE tix LIVE NO...  
1  spotty but pretty good. the web is actually wo...  
2  I’m in a jungle in Brazil! 🍃 https://t.co/xFa...  
3  What’s good with #SpringFling Presale prep?\n\...  
4  .@damitscott \nI can’t stand it when they’re o...

In [53]:
import re

In [54]:
kaskade_words=[]
for i in kaskade_tweets['text']:
    kaskade_words.append(re.sub(r"http\S+","", i))
kaskade_words

['You + me + a #SpringFling! PRESALE tix LIVE NOW. \n for password + links ',
 'spotty but pretty good. the web is actually world wide, as it turns out. ',
 'I\xe2\x80\x99m in a jungle in Brazil! \xf0\x9f\x8d\x83 ',
 'What\xe2\x80\x99s good with #SpringFling Presale prep?\n\nHave u handled it? Got your snacks, your screens up + the password?\n\nGoing live in 1 hour (ish)',
 '.@damitscott \nI can\xe2\x80\x99t stand it when they\xe2\x80\x99re off. It\xe2\x80\x99s like an OCD itch that can\xe2\x80\x99t be scratched. This video is \xf0\x9f\x91\x8d\xf0\x9f\x8f\xbb',
 '#SpringFling\nSix Cities.\nOne Presale date: WED. 03/01\n9 AM PST / NOON EST\n to sign up!',
 'but funny. ',
 'it was but WE HAVE CHANGED IT. Way less confusing. so presale tix go on sale\nWeds. 03/01 9 AM Pacific/Noon Eastern ',
 '#SpringFling presale live TOMORROW\n9 AM Pacific\nNOON Eastern\n ',
 "When Kaskade announces he's playing in six cities, but yours isn't on the list. ",
 '.@deadmau5 @imchasebrooks \nMaybe so, but w

In [55]:
retweeters1=pd.DataFrame(tweets2)
retweeters2=pd.DataFrame(tweets3)
retweeters3=pd.DataFrame(tweets4)
retweeters4=pd.DataFrame(tweets5)
retweeters5=pd.DataFrame(tweets6)
retweeters6=pd.DataFrame(tweets7)
retweeters7=pd.DataFrame(tweets8)
retweeters8=pd.DataFrame(tweets9)
retweeters9=pd.DataFrame(tweets10)
retweeters10=pd.DataFrame(tweets11)

In [56]:
frames=[retweeters1,retweeters2,
    retweeters3,retweeters4,
    retweeters5,retweeters6,
    retweeters7,retweeters8,
    retweeters9,retweeters10]

In [57]:
all_retweets=pd.concat(frames)
all_retweets.head()

created_at              id_str  retweet_count screen_name  \
0 2017-02-28 16:47:36  836618869989081090              4  Kaskade_MX   
1 2017-02-28 16:47:17  836618790490226688             12  Kaskade_MX   
2 2017-02-28 16:47:04  836618735330930688             67  Kaskade_MX   
3 2017-02-25 06:31:54  835376757335343106             27  Kaskade_MX   
4 2017-02-25 06:31:50  835376744005795840             14  Kaskade_MX   

                                                text  
0  RT @fairchild_music: Something big coming soon...  
1  RT @Arkade: TITAN - @fairchild_music \n3.3.17 ...  
2  RT @kaskade: In case you were working or somet...  
3  RT @kaskade: Full on summer vibes here in Braz...  
4  RT @Arkade: Risqué - @Thekissermusic \n💋OUT N...

In [58]:
clean_text=[]
for i in all_retweets['text']:
    clean_text.append(re.sub(r"http\S+","", i))

In [59]:
no_RT=[]
for i in clean_text:
    no_RT.append(re.sub(r"RT", "", i))

In [60]:
retweet_text=pd.DataFrame(no_RT, )

In [61]:
all_retweets['text']=retweet_text
all_retweets.head()

created_at              id_str  retweet_count screen_name  \
0 2017-02-28 16:47:36  836618869989081090              4  Kaskade_MX   
1 2017-02-28 16:47:17  836618790490226688             12  Kaskade_MX   
2 2017-02-28 16:47:04  836618735330930688             67  Kaskade_MX   
3 2017-02-25 06:31:54  835376757335343106             27  Kaskade_MX   
4 2017-02-25 06:31:50  835376744005795840             14  Kaskade_MX   

                                                text  
0   @fairchild_music: Something big coming soon. ...  
1        @Arkade: TITAN - @fairchild_music \n3.3.17   
2   @kaskade: In case you were working or somethi...  
3   @kaskade: Full on summer vibes here in Brazil...  
4   @Arkade: Risqué - @Thekissermusic \n💋OUT NOW...

In [97]:
all_retweets.to_pickle('retweets.pkl')

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords

In [87]:
stopset = set(stopwords.words('english'))
stopset.update(stopwords.words('spanish'))
stopset.update(['love', 'like', 'new', 'amp', 'get', 'one', 'see', 'si', 'today', 'tonight',
               'go','want','good', 'back', 'thank', 'us', 'day','people','make',
                'much', 'know',])

In [88]:
retweets_str=','.join(no_RT)

In [89]:
vectorizer=TfidfVectorizer(stop_words=stopset,use_idf=True, ngram_range=(1,3))
X=vectorizer.fit_transform([retweets_str])

In [90]:
lsa=TruncatedSVD(n_components=1, n_iter=100, random_state=42)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=1, n_iter=100,
       random_state=42, tol=0.0)

In [98]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:20]
    print("Kaskade %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

Kaskade 0:
kaskade
hakanmemories
music
time
thechainsmokers
night
last
happy
remix
thanks
show
check
hakanmemories hakanmemories
live
spotify
year
track
got
spinninrecords
video
 
